In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import torch
import os

In [2]:
data = pd.read_csv("credit_card.csv")
data.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0


In [3]:
input_vars = data[["amount","oldbalanceOrg","newbalanceOrig","oldbalanceDest"]]
target_vars = data[["isFraud"]]
print(len(input_vars),len(target_vars))

6362620 6362620


In [4]:
input_ar = np.array(input_vars)
target_ar = np.array(target_vars)

print(input_ar.shape)
print(target_ar.shape)

(6362620, 4)
(6362620, 1)


In [5]:
from sklearn.model_selection import train_test_split

In [6]:
inputs_train,inputs_test,targets_train,targets_test = train_test_split(input_ar, target_ar, test_size=0.20)

In [7]:
inputs_train = torch.from_numpy(inputs_train.astype(np.float32))
inputs_test = torch.from_numpy(inputs_test.astype(np.float32))
targets_train = torch.from_numpy(targets_train.astype(np.float32))
targets_test = torch.from_numpy(targets_test.astype(np.float32))

In [8]:
print(inputs_train.shape)
print(inputs_test.shape)
print(targets_train.shape)
print(targets_test.shape)

torch.Size([5090096, 4])
torch.Size([1272524, 4])
torch.Size([5090096, 1])
torch.Size([1272524, 1])


In [9]:
class LogisticRegression(nn.Module):
    def __init__(self,no_of_features):
        super(LogisticRegression, self).__init__()
        self.linear1 = nn.Linear(no_of_features,10)
        self.relu = nn.ReLU()
        self.linear2 = nn.Linear(10,1)
        self.sigmoid = nn.Sigmoid()
        

    def forward(self, targets_train ):
        out = self.linear1(targets_train)
        out  = self.relu(out)
        out = self.linear2(out)
        out = self.sigmoid(out)
        
        return out

model = LogisticRegression(no_of_features=4)

In [10]:
loss_fn = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [14]:
num_epochs = 15*5

In [12]:
from tqdm.notebook import tqdm

In [15]:
loss_hist = []
acc_hist = []
for epoch in tqdm(range(num_epochs)):
    targets_preds = model(inputs_train)
    loss = loss_fn(targets_preds,targets_train)

    loss.backward()

    optimizer.step()
    optimizer.zero_grad()

    if(epoch+1)%3 == 0:
        with torch.no_grad():
            targets_preds = model(inputs_test)
            targets_preds_cls = targets_preds.round()
            acc = targets_preds_cls.eq(targets_test).sum() / float(inputs_test.shape[0])
            loss_hist.append(loss.item())
            acc_hist.append(acc)
            print(f'Epoch: {epoch+1}, Loss: {loss.item():.4f}, ACC: {acc}')

  0%|          | 0/75 [00:00<?, ?it/s]

Epoch: 3, Loss: 37.8088, ACC: 0.626918613910675
Epoch: 6, Loss: 35.9997, ACC: 0.6493582725524902
Epoch: 9, Loss: 29.2198, ACC: 0.7380701899528503
Epoch: 12, Loss: 21.0895, ACC: 0.8108522891998291
Epoch: 15, Loss: 13.0948, ACC: 0.8976773619651794
Epoch: 18, Loss: 5.4617, ACC: 0.960547685623169
Epoch: 21, Loss: 3.3618, ACC: 0.9691824913024902
Epoch: 24, Loss: 2.2319, ACC: 0.979430615901947
Epoch: 27, Loss: 1.7901, ACC: 0.9830077886581421
Epoch: 30, Loss: 1.4893, ACC: 0.985869824886322
Epoch: 33, Loss: 1.2254, ACC: 0.9883350133895874
Epoch: 36, Loss: 0.9961, ACC: 0.9906123280525208
Epoch: 39, Loss: 0.7577, ACC: 0.9930539727210999
Epoch: 42, Loss: 0.5375, ACC: 0.995206356048584
Epoch: 45, Loss: 0.3694, ACC: 0.9969580173492432
Epoch: 48, Loss: 0.2141, ACC: 0.998092770576477
Epoch: 51, Loss: 0.1702, ACC: 0.998456597328186
Epoch: 54, Loss: 0.1408, ACC: 0.9986915588378906
Epoch: 57, Loss: 0.1296, ACC: 0.9987339973449707
Epoch: 60, Loss: 0.1296, ACC: 0.9987339973449707
Epoch: 63, Loss: 0.1296, 

In [17]:
torch.save(model,"credit_card_4-1.pth")

In [30]:
def predict_value(inputs):
    if len(inputs)==4:
        inputs = np.array(inputs)
        inputs = torch.from_numpy(inputs.astype(np.float32))
        main_model = torch.load("credit_card_4-1.pth")
        pred = main_model(inputs)
        value = pred.item()
        if value < 0.5:
            print("No Fraud")
        else:
            print("Its a Fraud")
    else:
        print("Length not matchd!")

In [31]:
input_l = [9839.64,170136.0,160296.36,0.0]
predict_value(input_l)

No Fraud
